# Structs

* A *struct* assembles several values of assorted types together into a single value, so you can deal with them as a unit. 
* Given a struct, you can read and modify its individual components. 
* A struct can have methods associated with it that operate on its components.

## Named-Field Structs

### Declaring a Struct

In [3]:
/// A rectangle of eight-bit grayscale pixels.
struct GrayscaleMap {
    pixels: Vec<u8>,
    size: (usize, usize)
}

### Creating a Struct Value

In [4]:
let width = 800;
let height = 600;

let image = GrayscaleMap {
    pixels: vec![0; width * height],
    size: (width, height)
};

In [8]:
fn create_image(size: (usize, usize)) -> GrayscaleMap {
    let pixels = vec![0; size.0 * size.1];
    GrayscaleMap{pixels, size} // struct expression
}

### Accessing Struct Fields

In [11]:
let image = create_image((800, 600));

assert!(image.size == (800, 600));
assert!(image.pixels.len() == image.size.0 * image.size.1);

### Visibility of Struct

* Structs are private by default, visible only in the module where they’re declared. 
* You can make a struct visible outside its module by prefixing its definition with `pub`

In [14]:
/// A rectangle of eight-bit grayscale pixels.
pub struct GrayscaleMap {
    pub pixels: Vec<u8>,
    pub size: (usize, usize)
}

## Tuple-Like Structs

## Tuple-Like Structs

* The second kind of struct type is called a tuple-like struct, because it resembles a tuple:

In [19]:
struct BoundingBox(usize, usize);

let image_bbox = BoundingBox(800, 600);

assert!(image_bbox.0 * image_bbox.1 == 480_000);

* Individual elements of a tuple-like struct may be public or not:

In [21]:
pub struct BoundingBox(pub usize, pub usize);

### Tuple-Like Struct as Newtypes

In [22]:
struct Ascii(Vec<u8>); // for stricter type checking

## Unit-Like Structs

* The third kind of struct is a little obscure: it declares a struct type with no elements at all:

In [26]:
struct Onesuch;

let o = Onesuch;

## Defining Methods - `impl`

In [39]:
pub struct Queue {
    older: Vec<char>,
    younger: Vec<char>
}

impl Queue {
    pub fn push(&mut self, c: char) {
        self.younger.push(c);
    }

    pub fn pop(&mut self) -> Option<char> {
        if self.older.is_empty() {
            if self.younger.is_empty() {
                return None;
            }

            use std::mem::swap;
            swap(&mut self.older, &mut self.younger);
            self.older.reverse();
        }

        self.older.pop()
    }

    pub fn is_empty(&self) -> bool {
        self.older.is_empty() && self.younger.is_empty()
    }
}

In [38]:
let mut q = Queue{ older: Vec::new(), younger: Vec::new() };

q.push('@');
q.push('$');
assert!(q.pop() == Some('@'));

q.push('*');
assert!(q.pop() == Some('$'));
assert!(q.pop() == Some('*'));
assert!(q.pop() == None);

assert!(q.is_empty());

In [41]:
impl Queue {
    pub fn split(self) -> (Vec<char>, Vec<char>) {
        (self.older, self.younger)
    }
}

In [43]:
let mut q = Queue { older: Vec::new(), younger: Vec::new() };
q.push('Q');
q.push('U');
assert!(q.pop() == Some('Q'));
q.push('X');

let (older, younger) = q.split();

assert_eq!(older, vec!['U']);
assert_eq!(younger, vec!['X']);

### `new` Method

* Constructor function - doesn't take `self` as an argument (**static function**)

In [44]:
impl Queue {
    pub fn new() -> Queue {
        Queue { older: Vec::new(), younger: Vec::new() }
    }
}

In [46]:
let mut q = Queue::new();

## Generic Structs

In [57]:
pub struct Stack<T> {
    items: Vec<T>
}

impl<T> Stack<T> {
    pub fn new() -> Stack<T> {
        Stack{ items: Vec::new() }
    }

    pub fn push(&mut self, item: T) {
        self.items.push(item)
    }

    pub fn pop(&mut self) -> Option<T> {
        self.items.pop()
    }

    pub fn is_empty(&self) -> bool {
        self.items.is_empty()
    }
}

* For static method calls, you can supply the type parameter explicitly using the turbofish `::<>` notation:

In [ ]:
let s = Stack::<char>::new();

* But in practice, Rust can figure it out for you:

In [59]:
let mut stack_1 = Stack::new();
let mut stack_2 = Stack::new();

stack_1.push("Hello");
stack_2.push(3.1415);

stack_1.push("World");
stack_2.push(2.71);

## Deriving Common Traits for Structs

* Structs are not copyable or comparable by default
* You can derive the `Copy` and `Clone` traits for a struct type, but only if all its fields are copyable or clonable
* You can derive the `PartialEq` and `Eq` traits for a struct type, but only if all its fields are comparable for equality
* You can derive `Debug` for a struct type, but only if all its fields are printable

In [3]:
#[derive(Copy, Clone, Debug, PartialEq)]
struct Point {
    x: f64,
    y: f64
}

In [14]:
let mut pt = Point{ x: 0.0, y: 1.0 };

let other_pt = pt;
assert!(pt == other_pt);

pt.x = 42.42;
assert!(pt != other_pt);

println!("pt = {:?}", pt);
println!("other_pt = {:?}", other_pt);

pt = Point { x: 42.42, y: 1.0 }
other_pt = Point { x: 0.0, y: 1.0 }
